In [1]:
# over welke periode gaat het dataframe: 10 jaar
# aggregratie op jaar niveau, eventueel later op week niveau

# welke X-en:
# PC4
# LON 
# LAT
# Res Regio
# Gemeente
# tem: hoe warm was het jaar?

# sjv_totaal
# factor_maand
# factor
# week_verbruik_kw
# laadpalen
# electrische auto's
# warmtepompen
# Opwekking zonneenergie
# Opwekking Windenergie
# Decentrale opwekgegevens
# Knelpunt indicatie (Provincie?)

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import seaborn as sns

In [3]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

In [4]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# Alle benodige ht5 dataframes inlezen

In [5]:
# decentrale_opwekkingsgegevens gegevens inlezen
df_opwek = pd.read_hdf('decentrale_opwekkingsgegevens_data.h5')
# kleinverbruikgegevens gegevens inlezen
df_verbruik = pd.read_hdf('kleinverbruikgegevens_data.h5')
# mapping van CBS buurt naar PC4
df_cbs_buurt_pc4 = pd.read_hdf('cbs_buurt_pc4.h5')
# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')

# Nieuw master dataframe maken

In [6]:
# maak nieuw dataframe met de jaren
df = pd.DataFrame(pd.date_range(start='1/1/2010', end='31/12/2020' , freq='Y'), columns=['DATUM']).sort_values(by='DATUM', ascending=False)
df['JAAR'] = df['DATUM'].dt.year

# Vebruik toevoegen aan dataframe

In [7]:
# df_verbruik is nog per pc6, dus eerst omzetten naar pc4
df_verbruik_pc4 = df_verbruik.groupby(['PC4','JAAR']
                                      , as_index=False
                                     ).agg({'SJV_TOTAAL': 'sum'
                                            ,'AANSLUITINGEN_AANTAL': 'sum'  
                                            ,'LEVERINGSRICHTING_PERC': 'mean'
                                           }
                                          )

# join df met df_verbruik
df = pd.merge(df, df_verbruik_pc4, on='JAAR', how='left')

# Opwek toevoegen aan dataframe

In [8]:
# jaar toevoegen
# 2020-07-01 verwijderen, de maand van de peildatum moet januari zijn
df_opwek = df_opwek[df_opwek['Peildatum'].dt.month == 1]
# 2020-01-01 wordt 2019, 2021-01-01 wordt 2020
df_opwek['JAAR'] = df_opwek['Peildatum'].dt.year - 1
# PC4 toevoegen
df_cbs_buurt_pc4 = df_cbs_buurt_pc4.rename(columns={'Buurt2020': 'CBS Buurtcode'})
df_opwek = pd.merge(df_opwek, df_cbs_buurt_pc4, on='CBS Buurtcode', how='left', suffixes=('','_2'))
# df_opwek group by PC4 en JAAR
df_opwek_pc4 = df_opwek.groupby(['PC4','JAAR'], as_index=False).agg({'Gemeente':'max'
                                                                     , 'Aantal aansluitingen in CBS-buurt': 'sum'
                                                                     , 'Aantal aansluitingen met opwekinstallatie':'sum'
                                                                     , 'Opgesteld vermogen':'sum'
                                                                    }
                                                                   )
# rename columns
df_opwek_pc4 = df_opwek_pc4.rename(columns={'Aantal aansluitingen in CBS-buurt': 'AANTAL AANSLUITINGEN IN PC4'
                                            , 'Gemeente': 'GEMEENTE'
                                            , 'Aantal aansluitingen met opwekinstallatie': 'AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE'
                                            , 'Opgesteld vermogen': 'OPGESTELD VERMOGEN'
                                           }
                                  )

# join df met df_opwek_pc4
df = pd.merge(df, df_opwek_pc4, on=['JAAR','PC4'], how='left')

# RES regio toevoegen aan dataframe

In [9]:
df = pd.merge(df, df_pc4_res, on=['PC4'], how='left')

# Save dataframe 

In [10]:
# data locatie om de ht5 file op te slaan
data_processed_location = '../processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [12]:
try:
    os.remove('master_data.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('master_data.h5')
store['master_data'] = df
store.close()